In [ ]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import numpy as np

import pytorch_lightning as pl
from pytorch_lightning.metrics.functional.classification import f1_score
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

## Data Loading

In [ ]:
import helper
data = helper.load_RWHAR()

from dataset import TimeSeriesDataset
dtset = TimeSeriesDataset(data)

weight = helper.dist(dtset, 7)

train, val = helper.split(dtset, val_pc = 0.25)
train_weight = helper.dist(train, 7)
val_weight = helper.dist(val, 7)
print(train_weight, val_weight)
train_iter = torch.utils.data.DataLoader(train, batch_size = 32, shuffle = True, num_workers = 8, pin_memory = True)
val_iter = torch.utils.data.DataLoader(val, batch_size = 32, num_workers = 8, pin_memory = True)

In [ ]:
# from DeepConvLSTM_model import DeepConvNet

# class Net(pl.LightningModule):
#     def __init__(self, in_channels = 3,hidden_size = 128, output_size = 7, classes_weight = None):
#         super(Net,self).__init__()
#         self.model = DeepConvNet(in_channels, hidden_size, output_size)
#         self.criterion = nn.CrossEntropyLoss(weight = classes_weight);
        
#     def forward(self, input_seq):
#         return self.model(input_seq)
    
#     def configure_optimizers(self):
#         return torch.optim.Adamax(self.parameters(), lr=0.0001)
    
#     def training_step(self, batch, batch_idx):
#         y_pred = self(batch["data"])
#         loss = self.criterion(y_pred, batch["label"])
#         self.log('train_loss', loss, on_step = False, on_epoch = True, prog_bar = False, logger = True)
#         self.log('train_f1_score', f1_score(y_pred, batch["label"], class_reduction = "micro"), on_step = False, on_epoch = True, prog_bar = True, logger = True)
#         return loss
    
#     def validation_step(self, batch, batch_idx):
#         y_pred = self(batch["data"])
#         loss = self.criterion(y_pred, batch["label"])
#         self.log('val_loss', loss, on_step = False, on_epoch = True, prog_bar = False, logger = True)
#         self.log('val_f1_score', f1_score(y_pred, batch["label"], class_reduction = "micro"), on_step = False, on_epoch = True, prog_bar = True, logger = True)
#         return loss

In [ ]:
from DeepConvLSTM_model import DeepConvNet
from Validation_model import Net
net = DeepConvNet(inchannels = 6, hidden_size = 256, output_size = 8)
model = Net(model = net, classes_weight = torch.tensor(train_weight, dtype = torch.float))

trainer = pl.Trainer(gpus=-1,
                     max_epochs=20,
                    #  limit_train_batches=0.1,
                    #  limit_val_batches=0.1,
                     log_every_n_steps = 200,
                     callbacks = [EarlyStopping(monitor = "val_f1_score", patience = 3, mode = "max")],
                     logger = TensorBoardLogger(save_dir = 'logs/', name = "PAMAP2_LSTM_model"),
                     )
trainer.fit(model, train_iter, val_iter)